In [3]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import ast


# Cấu hình kết nối (bạn thay thông tin cho đúng)
db_user = 'postgres'
db_password = 'changethis'
db_host = 'localhost'
db_port = '5432'
db_name = 'app'

# Tạo connection
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


In [12]:

df = pd.read_csv('./movies_metadata.csv', low_memory=False)
df = df.drop([19730, 29503, 35587])

columns_keep = [
    'id', 'title', 'original_title', 'belongs_to_collection', 'original_language', 'release_date', 'status',
    'overview', 'tagline', 'adult', 'popularity', 'homepage', 'poster_path',
    'runtime', 'budget', 'revenue', 'vote_average', 'vote_count'
]
df = df[columns_keep]

df = df.replace({np.nan: None})

df = df.drop_duplicates(subset=['id'], keep='first')

df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce').dt.date
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')
df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce').fillna(0).astype('Int64')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce').fillna(0).astype('Int64')
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce').fillna(0).astype('Int64')
df['vote_average'] = pd.to_numeric(df['vote_average'], errors='coerce')
df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce').fillna(0).astype('Int64')
df['belongs_to_collection'] = df['belongs_to_collection'].fillna('').apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x != '' else np.nan
).apply(
    lambda x: x['name'] if isinstance(x, dict) and 'name' in x else ''
)


# 6. Insert vào database
with engine.begin() as connection:
    df.to_sql('stg_movie_metadata', con=connection, if_exists='append', index=False, method='multi')

print("✅ Insert thành công!")


✅ Insert thành công!


In [23]:

md = pd.read_csv('./movies_metadata.csv', low_memory=False)
md = md.drop([19730, 29503, 35587])

genres = md[['id', 'genres']].copy()

genres['genres'] = genres['genres'].apply(ast.literal_eval)

genres.rename(columns={'id': 'movie_id'}, inplace=True)

rows = []
for idx, row in genres.iterrows():
    movie_id = row['movie_id']
    for genre in row['genres']:
        rows.append({'movie_id': movie_id, 'genre': genre['name']})

genres = pd.DataFrame(rows)
genres["key_id"] = genres.index + 1

with engine.begin() as connection:
    genres[["key_id", "movie_id", "genre"]].to_sql('stg_genre', con=connection, if_exists='append', index=False, method='multi')

print("✅ Insert bảng movie_genres thành công!")

✅ Insert bảng movie_genres thành công!


In [32]:
md = pd.read_csv('./movies_metadata.csv', low_memory=False)
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')
keywords = pd.read_csv('./keywords.csv')
keywords['id'] = keywords['id'].astype('int')
md = md.merge(keywords, on='id')
md['keywords'] = md['keywords'].apply(ast.literal_eval)
md['keywords'] = md['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

rows = []
for idx, row in md.iterrows():
    movie_id = row['id']
    for keyword in row['keywords']:
        rows.append({'movie_id': movie_id, 'keyword': keyword})

keywords = pd.DataFrame(rows)
keywords["key_id"] = keywords.index + 1

with engine.begin() as connection:
    keywords[["key_id", "movie_id", "keyword"]].to_sql('stg_keyword', con=connection, if_exists='append', index=False, method='multi')

print("✅ Insert bảng stg_keyword thành công!")

✅ Insert bảng movie_genres thành công!


In [8]:
md = pd.read_csv('./movies_metadata.csv', low_memory=False)
md = md.drop([19730, 29503, 35587])

credits = pd.read_csv('./credits.csv')
md['id'] = md['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md = md.merge(credits, on='id')

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

md['cast'] = md['cast'].apply(ast.literal_eval)
md['cast'] = md['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['cast'] = md['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
md['crew'] = md['crew'].apply(ast.literal_eval)
md['director'] = md['crew'].apply(get_director).fillna('')

md[["id", 'cast', 'crew', 'director']].head()
md['movie_id'] = md['id']

rows = []
for idx, row in md.iterrows():
    movie_id = row['id']
    for cast in row['cast']:
        rows.append({'movie_id': movie_id, 'name': cast, 'role': 'cast'})
    rows.append({'movie_id': movie_id, 'name': row['director'], 'role': 'director'})


produce = pd.DataFrame(rows)
produce["key_id"] = produce.index + 1

with engine.begin() as connection:
    produce[["key_id", "movie_id", "name", "role"]].to_sql('stg_cast', con=connection, if_exists='append', index=False, method='multi')

print("✅ Insert bảng stg_cast thành công!")

✅ Insert bảng stg_cast thành công!


In [2]:
rating = pd.read_csv('./ratings.csv')
rating.rename(columns={"movieId": "movie_id", "userId": "user_id"}, inplace=True)
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')
rating["key_id"] = rating.index + 1
rating.head()

,user_id,movie_id,rating,timestamp,key_id
0,1,110,1.0,2015-03-09 22:52:09,1
1,1,147,4.5,2015-03-09 23:07:15,2
2,1,858,5.0,2015-03-09 22:52:03,3
3,1,1221,5.0,2015-03-09 22:52:26,4
4,1,1246,5.0,2015-03-09 22:52:36,5


In [6]:
links = pd.read_csv('./links.csv')
links.rename(columns={"movieId": "movie_id", "imdbId": "imdb_id", "tmdbId": "tmdb_id"}, inplace=True)

with engine.begin() as connection:
    links.to_sql('stg_links', con=connection, if_exists='append', index=False, method='multi')

print("✅ Insert stg_links thành công!")


✅ Insert stg_links thành công!


In [2]:

# Query data from database
with engine.connect() as connection:
    # Query movies
    movies_query = "SELECT id, title, vote_average, vote_count FROM stg_movie_metadata"
    movies_df = pd.read_sql(movies_query, connection)

    # Query genres
    genres_query = "SELECT movie_id, genre FROM stg_genre"
    genres_df = pd.read_sql(genres_query, connection)

# Print data summary
print(f"Loaded {len(movies_df)} movies and {len(genres_df)} genre entries.")
print("Movies sample:")
print(movies_df.head())
print("Genres sample:")
print(genres_df.head())


Loaded 45433 movies and 91094 genre entries.
Movies sample:
       id                title  vote_average  vote_count
0  258945  Anything Can Happen           4.5           2
1    2124       Color of Night           5.4         117
2    1444              Junebug           6.5          66
3  104374    Woman of the Lake           6.9           7
4    5528           The Chorus           7.5         553
Genres sample:
   movie_id      genre
0       862  Animation
1       862     Comedy
2       862     Family
3      8844  Adventure
4      8844    Fantasy


In [3]:
import sqlalchemy
from datetime import datetime

# Calculate vote_average, vote_count_80th, vote_count_90th, vote_count_99th for each genre
unique_genres = genres_df['genre'].unique()
stats = []

for genre in unique_genres:
    if not genre:
        continue
    genre_movies = pd.merge(
        genres_df[genres_df['genre'] == genre],
        movies_df,
        left_on='movie_id',
        right_on='id'
    )
    vote_counts = genre_movies[genre_movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = genre_movies[genre_movies['vote_average'].notnull()]['vote_average'].astype('float')

    vote_average = vote_averages.mean() if not vote_averages.empty else 0
    vote_count_80th = vote_counts.quantile(0.80) if not vote_counts.empty else 0
    vote_count_90th = vote_counts.quantile(0.90) if not vote_counts.empty else 0
    vote_count_99th = vote_counts.quantile(0.99) if not vote_counts.empty else 0

    stats.append({
        'genre': genre,
        'vote_average': vote_average,
        'vote_count_80th': vote_count_80th,
        'vote_count_90th': vote_count_90th,
        'vote_count_99th': vote_count_99th,
        'updated_at': datetime.now()
    })

# Convert to DataFrame
stats_df = pd.DataFrame(stats)
print("Calculated stats for each genre:")
print(stats_df)

# Update stg_vote_extended table
with engine.connect() as connection:
    with connection.begin():
        # Delete existing records
        connection.execute(sqlalchemy.text("DELETE FROM stg_vote_extended"))

        # Insert new records
        for _, row in stats_df.iterrows():
            insert_query = """
            INSERT INTO stg_vote_stats (genre, vote_average, vote_count_80th, vote_count_90th, vote_count_99th, updated_at)
            VALUES (:genre, :vote_average, :vote_count_80th, :vote_count_90th, :vote_count_99th, :updated_at)
            """
            connection.execute(sqlalchemy.text(insert_query), {
                'genre': row['genre'],
                'vote_average': row['vote_average'],
                'vote_count_80th': row['vote_count_80th'],
                'vote_count_90th': row['vote_count_90th'],
                'vote_count_99th': row['vote_count_99th'],
                'updated_at': row['updated_at']
            })

print("Updated stg_vote_extended table successfully.")

# Verify data
with engine.connect() as connection:
    verify_query = "SELECT genre, vote_average, vote_count_80th, vote_count_90th, vote_count_99th, updated_at FROM stg_vote_stats"
    verify_df = pd.read_sql(verify_query, connection)
    print("Current stg_vote_extended table:")
    print(verify_df)

Calculated stats for each genre:
              genre  vote_average  vote_count_80th  vote_count_90th  \
0         Animation      6.275181            150.2            453.6   
1            Comedy      5.715741             71.0            207.0   
2            Family      5.753394            151.2            509.3   
3         Adventure      5.686070            282.0            994.5   
4           Fantasy      5.787722            209.6            806.8   
5           Romance      5.802658             58.0            168.0   
6             Drama      5.905221             53.0            155.0   
7            Action      5.584167            123.0            491.0   
8             Crime      5.878407            121.0            367.4   
9          Thriller      5.616713            137.0            388.7   
10           Horror      5.230394             80.0            207.8   
11          History      6.154220             72.0            184.6   
12  Science Fiction      5.342895           

In [2]:
# Query data
with engine.connect() as connection:
    # Get movies
    movies_query = "SELECT id, title, vote_average, vote_count FROM stg_movie_metadata"
    movies_df = pd.read_sql(movies_query, connection, dtype={'id': 'int32', 'vote_average': 'float32', 'vote_count': 'int32'})

    # Get genres
    genres_query = "SELECT key_id, movie_id, genre FROM stg_genre"
    genres_df = pd.read_sql(genres_query, connection, dtype={'key_id': 'int32', 'movie_id': 'int32'})

    # Get stats from stg_vote_extended
    stats_query = "SELECT genre, vote_average, vote_count_80th, vote_count_90th, vote_count_99th FROM stg_vote_extended"
    stats_df = pd.read_sql(stats_query, connection)

# Print data summary
print(f"Loaded {len(movies_df)} movies, {len(genres_df)} genre entries, {len(stats_df)} genre stats.")
print("Movies sample:\n", movies_df.head())
print("Genres sample:\n", genres_df.head())
print("Stats sample:\n", stats_df.head())

# Calculate WR percentiles for each genre
total_df = pd.merge(
    movies_df,
    genres_df,
    left_on='id',
    right_on='movie_id'
)

total_df = pd.merge(
    total_df,
    stats_df,
    on='genre'
)

total_df.head()


Loaded 45433 movies, 91094 genre entries, 20 genre stats.
Movies sample:
        id              title  vote_average  vote_count
0     176                Saw           7.2        2255
1  333318  Scherzo Diabolico           6.0           8
2  362185           Operator           5.2          21
3   45325       Tom and Huck           5.4          45
4   40047              Elvis           6.4          15
Genres sample:
    key_id  movie_id      genre
0       1       862  Animation
1       2       862     Comedy
2       3       862     Family
3       4      8844  Adventure
4       5      8844    Fantasy
Stats sample:
        genre  vote_average  vote_count_80th  vote_count_90th  vote_count_99th
0  Animation      6.275181            150.2            453.6          4272.76
1     Comedy      5.715741             71.0            207.0          1743.57
2     Family      5.753394            151.2            509.3          4672.74
3  Adventure      5.686070            282.0            994.5       

,id,title,vote_average_x,vote_count,key_id,movie_id,genre,vote_average_y,vote_count_80th,vote_count_90th,vote_count_99th
0,176,Saw,7.2,2255,19703,176,Horror,5.230394,80.0,207.8,1566.40
1,176,Saw,7.2,2255,19704,176,Mystery,5.820470,108.8,357.8,2813.46
2,176,Saw,7.2,2255,19705,176,Crime,5.878407,121.0,367.4,3000.76
3,333318,Scherzo Diabolico,6.0,8,83497,333318,Comedy,5.715741,71.0,207.0,1743.57
4,333318,Scherzo Diabolico,6.0,8,83498,333318,Horror,5.230394,80.0,207.8,1566.40


In [6]:
v = total_df['vote_count']
R = total_df['vote_average_x']
m_80th = total_df['vote_count_80th']
m_90th = total_df['vote_count_90th']
m_99th = total_df['vote_count_99th']
C = total_df['vote_average_y']

total_df['wr_80th'] = (R * v / (v + m_80th)) + (C * m_80th / (v + m_80th))
total_df['wr_90th'] = (R * v / (v + m_90th)) + (C * m_90th / (v + m_90th))
total_df['wr_99th'] = (R * v / (v + m_99th)) + (C * m_99th / (v + m_99th))

total_df.head()

,id,title,vote_average_x,vote_count,key_id,movie_id,genre,vote_average_y,vote_count_80th,vote_count_90th,vote_count_99th,wr_80th,wr_90th,wr_99th
0,176,Saw,7.2,2255,19703,176,Horror,5.230394,80.0,207.8,1566.40,7.132519,7.033813,6.392654
1,176,Saw,7.2,2255,19704,176,Mystery,5.820470,108.8,357.8,2813.46,7.136503,7.011085,6.434234
2,176,Saw,7.2,2255,19705,176,Crime,5.878407,121.0,367.4,3000.76,7.132696,7.014844,6.445441
3,333318,Scherzo Diabolico,6.0,8,83497,333318,Comedy,5.715741,71.0,207.0,1743.57,5.744527,5.726318,5.717039
4,333318,Scherzo Diabolico,6.0,8,83498,333318,Horror,5.230394,80.0,207.8,1566.40,5.300358,5.258924,5.234304


In [14]:
from sqlalchemy.types import Integer, Text, Float


temp_table = 'temp_stg_genre_update'

# Use SQLAlchemy types in the dtype argument
total_df[['movie_id', 'genre', 'wr_80th', 'wr_90th', 'wr_99th']].to_sql(
    temp_table,
    engine,
    if_exists='replace',
    index=False,
    dtype={
        'movie_id': Integer(),
        'genre': Text(),
        'wr_80th': Float(),
        'wr_90th': Float(),
        'wr_99th': Float()
    }
)



94

In [ ]:

# Update stg_genre using temporary table
with engine.connect() as connection:
    with connection.begin():
        update_query = """
            UPDATE stg_genre g
            SET
                wr_80th = t.wr_80th,
                wr_90th = t.wr_90th,
                wr_99th = t.wr_99th
            FROM temp_stg_genre_update t
            WHERE g.movie_id = t.movie_id
            AND g.genre = t.genre
        """
        result = connection.execute(update_query)
        print(f"Updated {result.rowcount} rows in stg_genre.")

        # Drop temporary table
        connection.execute(f"DROP TABLE IF EXISTS {temp_table}")

# Verify update
with engine.connect() as connection:
    verify_query = """
        SELECT movie_id, genre, wr_80th, wr_90th, wr_99th
        FROM stg_genre
        WHERE wr_80th IS NOT NULL OR wr_90th IS NOT NULL OR wr_99th IS NOT NULL
        ORDER BY movie_id, genre
        LIMIT 5
    """
    verify_df = pd.read_sql(verify_query, connection)
    print("\nSample updated rows in stg_genre:")
    print(verify_df.to_string(index=False))

In [10]:
from sentence_transformers import SentenceTransformer
import pickle  # Để lưu dictionary

query = """
SELECT
    smm.id,
    NULLIF(CONCAT_WS('.', smm.title, smm.belongs_to_collection), '') AS title_vector,
    NULLIF(CONCAT_WS('.', smm.overview, smm.tagline), '') AS content_vector,
    NULLIF(CONCAT_WS(',', smm.keywords, sg.genres), '') AS type_vector,
    NULLIF(CONCAT_WS(',', sc.cast), '') AS people_vector
FROM stg_movie_metadata smm
LEFT JOIN (
    SELECT movie_id, STRING_AGG(genre, ',') AS genres
    FROM stg_genre
    GROUP BY movie_id
) sg ON smm.id = sg.movie_id
LEFT JOIN (
    SELECT movie_id, STRING_AGG(concat(role, ' ', name), '.') AS cast
    FROM stg_cast
    GROUP BY movie_id
) sc ON smm.id = sc.movie_id
"""

df = pd.read_sql(query, engine)

In [11]:
df.to_csv("movie_content.csv")

In [12]:
df

,id,title_vector,content_vector,type_vector,people_vector
0,110457,Windbag the Sailor.,Will Hay plays a bragging sea captain whose ma...,Comedy,cast Norma Varden.cast Will Hay.cast Moore Mar...
1,174751,Jane Got a Gun.,After her outlaw husband returns home shot wit...,"Drama,Action,Western",cast Joel Edgerton.cast Ewan McGregor.cast Nat...
2,26379,Paa.,He suffers from a progeria like syndrome. Ment...,"Drama,Family,Foreign",cast Amitabh Bachchan.cast Vidya Balan.cast Ab...
3,89877,The Reunion.Klassefesten (Samling),'The Renion' is a new Danish comedy starring T...,Comedy,cast Troels Lyby.cast Nicolaj Kopernikus
4,151687,Charlie Muffin.,"Charlie Muffin, top British Intelligence opera...","Thriller,TV Movie,Drama",cast David Hemmings.cast Sam Wanamaker.cast Je...
...,...,...,...,...,...
45428,126947,Fraternity Row.,Director Thomas J. Tobin's 1977 drama about co...,Drama,cast Scott Newman.cast Peter Fox.cast Nancy Mo...
45429,73526,The Doe Boy.,A Cherokee boy is a haemophiliac in a culture ...,"Drama,Romance",cast James Duval.director Randy Redroad.cast K...
45430,141976,Mistletoe Over Manhattan.,"Rebecca, Mrs. Claus (Tedde Moore) is worried t...","TV Movie,Family",cast Tedde Moore.cast Tricia Helfer.cast Greg ...
45431,70712,Te presento a Laura.,This is the story of 23-year-old Laura who liv...,"Romance,Comedy,Drama",cast Eugenio Derbez.cast Martha Higareda.direc...


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

embedding_dict = {}

for _, row in df.iterrows():
    item = {
        "title_vector": model.encode(row['title_vector'] or '', normalize_embeddings=True),
        "content_vector": model.encode(row['content_vector'] or '', normalize_embeddings=True),
        "type_vector": model.encode(row['type_vector'] or '', normalize_embeddings=True),
    }
    embedding_dict[row['id']] = item


with open('movie_embeddings.pkl', 'wb') as f:
    pickle.dump(embedding_dict, f)


# vector embedding

In [13]:
import pickle
import numpy as np
import faiss


with open('../vector-embedding/movie_embedding.pkl', 'rb') as f:
    embedding_dict = pickle.load(f)

ids = []
title_vectors = []
content_vectors = []
type_vectors = []
people_vectors = []

for movie_id, vecs in embedding_dict.items():
    ids.append(movie_id)
    title_vectors.append(vecs['title_vector'])
    content_vectors.append(vecs['content_vector'])
    type_vectors.append(vecs['type_vector'])
    people_vectors.append(vecs['people_vector'])

title_vectors = np.array(title_vectors).astype('float32')
content_vectors = np.array(content_vectors).astype('float32')
type_vectors = np.array(type_vectors).astype('float32')
people_vectors = np.array(people_vectors).astype('float32')

# Kích thước vector
dim = title_vectors.shape[1]

# Tạo FAISS index sử dụng cosine similarity = inner product vì vector đã được chuẩn hóa
index_title = faiss.IndexFlatIP(dim)
index_content = faiss.IndexFlatIP(dim)
index_type = faiss.IndexFlatIP(dim)
index_people = faiss.IndexFlatIP(dim)

index_title.add(title_vectors)
index_content.add(content_vectors)
index_type.add(type_vectors)
index_people.add(people_vectors)

# Lưu mapping từ FAISS index ID -> movieId thực
faiss_id_to_movieid = {i: mid for i, mid in enumerate(ids)}

faiss.write_index(index_title, '../vector-embedding/faiss_title.index')
faiss.write_index(index_content, '../vector-embedding/faiss_content.index')
faiss.write_index(index_type, '../vector-embedding/faiss_type.index')
faiss.write_index(index_people, '../vector-embedding/faiss_people.index')

# Cũng lưu mapping
with open('../vector-embedding/faissid_to_movieid.pkl', 'wb') as f:
    pickle.dump(faiss_id_to_movieid, f)


In [4]:
from sentence_transformers import SentenceTransformer
import pickle  # Để lưu dictionary

query = """
select * from stg_rating
"""

df_ratings = pd.read_sql(query, engine)

D:\PycharmProjects\movie-recomendation-system\backend\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df_ratings.head()

,key_id,user_id,movie_id,rating,timestamp
0,1,1,110,1.0,1425941529
1,2,1,147,4.5,1425942435
2,3,1,858,5.0,1425941523
3,4,1,1221,5.0,1425941546
4,5,1,1246,5.0,1425941556


In [6]:
df_ratings.to_csv("movie_ratings.csv")